In [ ]:
import pandas as pd

judges_url = 'https://www.fjc.gov/sites/default/files/history/federal-judicial-service.csv'

judges_df_download = pd.read_csv(judges_url)

In [ ]:
# Fix nominating_party values that start with "None"
# If there is no nominating party (usually because of a recess appointment that did not result in a commission),
# use the last nominating party
def fix_nominating_party(row):
    if row['nominating_party'].startswith("None"):
        previous_rows = judges_df[(judges_df['nid'] == row['nid']) & (judges_df['start_date'] < row['start_date'])]
        if not previous_rows.empty:
            last_valid_row = previous_rows[~previous_rows['nominating_party'].str.startswith("None")].sort_values('start_date', ascending=False).head(1)
            if not last_valid_row.empty:
                return last_valid_row['nominating_party'].values[0]
    return row['nominating_party']

# Preserve the original to avoid requesting the data again
judges_df = judges_df_download.copy()

judges_df.columns = judges_df.columns.str.lower().str.replace(' ', '_').str.replace('/', '_').str.replace(',', '_')

judges_df['start_date'] = judges_df['recess_appointment_date'].fillna(judges_df['commission_date'])
judges_df['nominating_party'] = judges_df['party_of_reappointing_president'].fillna(judges_df['party_of_appointing_president']).fillna("George Washington")

judges_df['nominating_party'] = judges_df.apply(fix_nominating_party, axis=1)

# Eliminate the courts that we do not want.
# Courts of Appeals started in 1911. So, our start date will be Jan. 1, 1912.
judges_df = judges_df[judges_df['court_type'].isin(["Supreme Court", "U.S. District Court", "U.S. Court of Appeals"])]

# Make sure date columns are datetime
judges_df['start_date'] = pd.to_datetime(judges_df['start_date'])
judges_df['termination_date'] = pd.to_datetime(judges_df['termination_date'])
judges_df['senior_status_date'] = pd.to_datetime(judges_df['senior_status_date'])

# Define the cutoff date
cutoff = pd.Timestamp('1912-01-01')

# Filter to judges serving on the cutoff date or later
judges_df = judges_df[
    (((judges_df['start_date'] <= cutoff) & ((judges_df['termination_date'].isna()) | (judges_df['termination_date'] >= cutoff)))
     | (judges_df['start_date'] > cutoff))
]

# Some helpful comparisons, at least for circuit courts:
# https://www.acslaw.org/judicial-nominations/change-in-court-composition/
# https://www.reddit.com/r/MapPorn/comments/18uxvdk/partisan_composition_of_every_district_courts/
# https://en.wikipedia.org/wiki/Judicial_appointment_history_for_United_States_federal_courts#:~:text=As%20of%20January%202%2C%202025%2C%20of%20the%20679%20district%20court,a%20majority%20in%204%20circuits.





In [ ]:
# Filter for active First Circuit judges
active_first_circuit = judges_df[
    (judges_df['court_name'] == 'U.S. Court of Appeals for the Second Circuit') &
    (judges_df['termination_date'].isna()) &
    (judges_df['senior_status_date'].isna())
]

# Count by nominating party
party_counts = active_first_circuit['nominating_party'].value_counts(dropna=False)
total = party_counts.sum()

# Create summary DataFrame
party_summary = party_counts.reset_index()
party_summary.columns = ['party', 'count']
party_summary['percentage'] = (party_summary['count'] / total * 100).round(1)

In [24]:
from maplibre import Layer, LayerType, Map, MapOptions
from maplibre.sources import GeoJSONSource
from maplibre.utils import df_to_geojson, geopandas_to_geojson
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point
import random

# --- 1. Generate Fake Home Data ---
NUM_HOMES = 100
CENTER_LAT = 21.3069  # Honolulu Latitude
CENTER_LON = -157.8583 # Honolulu Longitude
RADIUS = 0.05 # Degree radius for random points

lats = []
lons = []
for_sale_status = []
ids = []

for i in range(NUM_HOMES):
    lat = CENTER_LAT + random.uniform(-RADIUS, RADIUS)
    lon = CENTER_LON + random.uniform(-RADIUS, RADIUS)
    for_sale = random.choice([True, False])

    lats.append(lat)
    lons.append(lon)
    for_sale_status.append(for_sale)
    ids.append(f"home_{i}")

# Create a Pandas DataFrame
df = pd.DataFrame({
    'id': ids,
    'latitude': lats,
    'longitude': lons,
    'for_sale': for_sale_status
})

# --- 2. Create GeoDataFrame ---
# Convert the DataFrame to a GeoDataFrame
gdf = gpd.GeoDataFrame(
    df,
    geometry=gpd.points_from_xy(df.longitude, df.latitude),
    crs="EPSG:4326" # WGS 84 coordinate system
)

# Convert to GEOJSON and as source
gdf_json1 = gdf.to_json()
with open("homes_data.geojson", "w") as file:
    file.write(gdf_json1)


#gdf_json2 = geopandas_to_geojson(gdf) # This one works because it requires dictionary


# homes = GeoJSONSource(data=geopandas_to_geojson(gdf))

# map_options = MapOptions(
#     center=(CENTER_LON, CENTER_LAT),
#     zoom=12,
#     hash=True,
# )

# m = Map(map_options)
# m.add_layer(
#     Layer(
#         id='homes1',
#         type=LayerType.CIRCLE,
#         source=homes,
#         paint={
#             "circle-color": ["match", ["get", "for_sale"], 1, 'red', 'blue'],
#             "circle-radius": 5,
#         },
#     )
# )

# out_filename = 'home.html'
# with open(out_filename, "w") as f:
#     f.write(m.to_html())
